In [1]:
# 실습에 필요한 라이브러리를 불러옵니다.
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import vocab
from torchtext import datasets
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# IMDB 데이터세트의 학습 세트를 불러옵니다.
train_dataset = datasets.IMDB(split=('train'))

aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:03<00:00, 22.6MB/s]


In [3]:
# 토크나이저 객체를 생성합니다.
tokenizer = get_tokenizer('basic_english')

In [4]:
# 단어별 누적 사용 빈도를 계산하기 위해 카운터 객체를 생성합니다.
counter = Counter()

# 학습 세트의 문장을 단어 단위로 토큰화하고 단어별 누적 사용 빈도를 계산합니다.
for (label, text) in train_dataset:
    # 문장을 단어 단위로 토큰화하고 단어별로 사용 빈도를 기록합니다.
    counter.update(tokenizer(text))

In [5]:
# 10번 이상 사용된 단어를 사용해서 단어장을 만듭니다.
vocabulary = vocab(counter, min_freq=10)
vocabulary.set_default_index(0)

In [6]:
# 텍스트를 정수 인코딩하는 람다 함수를 정의합니다.
text_transform = lambda x: [vocabulary[token] for token in tokenizer(x)]

# 레이블을 정숫값으로 치환하는 람다 함수를 정의합니다.
label_transform = lambda x: 1 if x == 'pos' else 0

In [7]:
# 방금 정의한 두 개의 람다 함수를 이용해 텍스트와 레이블을 전처리하는 함수를 정의합니다.
def preprocessing(batch):
    label_list, text_list = [], []
    
    # 람다 함수를 사용해서 배치값을 차례대로 변환합니다.
    for (_label, _text) in batch:
        # 레이블에 람다 함수를 적용합니다.
        label_list.append(label_transform(_label))
        # 텍스트에 람다 함수를 적용합니다.
        text_list.append(torch.tensor(text_transform(_text)))
        
    # 가장 긴 문장을 기준으로 정수 인코딩 된 문장의 길이를 통일합니다.
    data = pad_sequence(text_list)
    target = torch.tensor(label_list)
    
    # 전처리 결과를 반환합니다.
    return data, target

In [8]:
# IMDB 데이터세트를 학습 세트와 테스트 세트로 나눠서 불러옵니다.
train_dataset, test_dataset = datasets.IMDB(split=('train', 'test'))

# preprocessing 함수를 적용하여 학습 세트 데이터로더와 테스트 세트 데이터로더를 만듭니다.
train_loader = DataLoader(list(train_dataset), batch_size=8, shuffle=True, collate_fn=preprocessing)
test_loader = DataLoader(list(test_dataset), batch_size=8, shuffle=False, collate_fn=preprocessing)

In [9]:
# LSTM 모델 클래스를 정의합니다.
class LSTM(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # 모델 구조를 정의합니다.
        self.embed = nn.Embedding(vocab_size, 16)
        self.cell = nn.LSTM(16, 16)
        self.fc = nn.Linear(16, 1)
        self.sigmoid = nn.Sigmoid()
        
    # 순전파를 정의합니다.
    def forward(self, X):
        out = self.embed(X)
        out, (hidden_state, cell_state) = self.cell(out)
        out = self.fc(hidden_state.view(-1, 16))
        out = self.sigmoid(out)
        return out

In [10]:
# 그래픽 카드 사용이 가능할 경우 그래픽 카드로 연산하도록 설정합니다.
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# LSTM 모델 객체를 생성합니다.
vocab_size = len(vocabulary)
model = LSTM(vocab_size).to(device)

# 이진 크로스 엔트로피(Binary Cross Entropy Error) 손실 함수 객체를 생성합니다.
criterion = nn.BCELoss().to(device)

# 아담 옵티마이저 객체를 생성합니다.
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [11]:
# 학습 함수를 정의합니다.
def train(model, criterion, optimizer, loader):
    # 현재 에포크의 오차와 정확도를 저장할 변수를 생성합니다.
    epoch_loss = 0
    epoch_acc = 0

    # 모델을 학습 모드로 설정합니다.
    model.train()
    # 배치 학습을 실행합니다.
    for X_batch, y_batch in loader:
        # 입력 데이터와 타깃을 준비합니다.
        X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
        # 기울기를 초기화합니다.
        optimizer.zero_grad()
        # 모델을 사용해 타깃을 추론합니다.
        hypothesis = model(X_batch)
        # 손실 함수로 오차를 계산합니다.
        loss = criterion(hypothesis, y_batch)
        # 기울기를 계산합니다.
        loss.backward()
        # 경사 하강법으로 가중치를 수정합니다.
        optimizer.step()
        # 정확도를 계산합니다.
        acc = ((hypothesis >= 0.5) == y_batch).float().mean()
        # 현재 배치의 오차와 정확도를 저장합니다.
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    # 현재 에포크의 오차와 정확도를 반환합니다.
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [12]:
# 평가 함수를 정의합니다.
def evaluate(model, criterion, optimizer, loader):
    # 현재 에포크의 오차와 정확도를 저장할 변수를 생성합니다.
    epoch_loss = 0
    epoch_acc = 0
    
    # 모델을 평가 모드로 설정합니다.
    model.eval()
    with torch.no_grad():
        # 배치 단위로 추론을 실행합니다.
        for X_batch, y_batch in loader:
            # 입력 데이터와 타깃을 준비합니다.
            X_batch, y_batch = X_batch.to(device), y_batch.to(device).float().view(-1, 1)
            # 모델을 사용해 타깃을 추론합니다.
            hypothesis = model(X_batch)
            # 손실 함수로 오차를 계산합니다.
            loss = criterion(hypothesis, y_batch)
            # 정확도를 계산합니다.
            acc = ((hypothesis >= 0.5) == y_batch).float().mean()
            # 현재 배치의 오차와 정확도를 저장합니다.
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    # 현재 에포크의 오차와 정확도를 반환합니다.
    return epoch_loss / len(loader), epoch_acc / len(loader)

In [13]:
# 25회에 걸쳐 모델을 학습합니다.
n_epochs = 25
for epoch in range(n_epochs):
    # 모델을 학습시킵니다.
    loss, acc = train(model, criterion, optimizer, train_loader)
    # 모델을 평가합니다.
    test_loss, test_acc = evaluate(model, criterion, optimizer, test_loader)
    
    # 현재 에포크의 학습 결과를 출력합니다.
    print('epoch: {}, loss: {:.3f}, acc: {:.2f}, test_loss: {:.3f}, test_acc: {:.3f}'.format(
        epoch, loss, acc, test_loss, test_acc
    ))

epoch: 0, loss: 0.693, acc: 0.50, test_loss: 0.692, test_acc: 0.509
epoch: 1, loss: 0.689, acc: 0.52, test_loss: 0.690, test_acc: 0.511
epoch: 2, loss: 0.685, acc: 0.52, test_loss: 0.688, test_acc: 0.514
epoch: 3, loss: 0.678, acc: 0.52, test_loss: 0.682, test_acc: 0.524
epoch: 4, loss: 0.673, acc: 0.53, test_loss: 0.679, test_acc: 0.527
epoch: 5, loss: 0.666, acc: 0.54, test_loss: 0.680, test_acc: 0.523
epoch: 6, loss: 0.659, acc: 0.55, test_loss: 0.684, test_acc: 0.538
epoch: 7, loss: 0.601, acc: 0.66, test_loss: 0.573, test_acc: 0.726
epoch: 8, loss: 0.528, acc: 0.76, test_loss: 0.572, test_acc: 0.749
epoch: 9, loss: 0.508, acc: 0.79, test_loss: 0.542, test_acc: 0.769
epoch: 10, loss: 0.617, acc: 0.61, test_loss: 0.649, test_acc: 0.549
epoch: 11, loss: 0.498, acc: 0.77, test_loss: 0.487, test_acc: 0.793
epoch: 12, loss: 0.530, acc: 0.75, test_loss: 0.620, test_acc: 0.679
epoch: 13, loss: 0.551, acc: 0.73, test_loss: 0.502, test_acc: 0.773
epoch: 14, loss: 0.411, acc: 0.83, test_loss